In [ ]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv("hotel_bookings.csv")

In [ ]:
df.head()

In [ ]:
#Changing the Month into the number representation. Example- Jan - 1, Feb - 2

df['arrival_date_month'] = pd.to_datetime(df['arrival_date_month'], format='%B').dt.month


In [ ]:
#Compling all the date related columns and making a column of datatype datetime

df['arrival_date'] = pd.to_datetime(df[['arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']].rename(columns={
    'arrival_date_year': 'year',
    'arrival_date_month': 'month',
    'arrival_date_day_of_month': 'day'
}))

In [ ]:
#Dropping the unwanted columns

df.drop(columns=['arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month'],inplace=True)

In [ ]:
df['stay_in_nights'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

In [ ]:
df.arrival_date.nunique()

In [ ]:
#Making a new dataframe for the predicting price
finaldf = pd.DataFrame()

In [ ]:
finaldf['arrival_date'] = df['arrival_date']

In [ ]:
finaldf['hotel'] = df['hotel']

In [ ]:
df['weekday'] = df['arrival_date'].dt.weekday

In [ ]:
finaldf['weekend'] = (df['weekday'] >= 5).astype(int)

In [ ]:
finaldf['adr'] = df['adr']

In [ ]:
finaldf["reserved_room_type"] = df["assigned_room_type"]

In [ ]:
finaldf.head(50)

In [ ]:
finaldf["nights_staying"] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

In [ ]:
finaldf["lead_time"] = df["lead_time"]

In [ ]:
finaldf = finaldf.sort_values(by=['arrival_date', 'lead_time'], ascending=[True, False])


In [ ]:
finaldf['adults'] = df['adults']
finaldf['children'] = df['children']
finaldf['babies'] = df['babies']
finaldf['meal']=df['meal']

In [ ]:
#Basically count is the column we have made which helps in predicting the price, count is the no. of the bookings done by the time that particular booking was made.

df['count'] = 0

# For each row, update the count for the respective stay duration
for index, row in df.iterrows():
    # Calculate the departure date (arrival date + stays in nights)
    stay_duration = row['stays_in_weekend_nights'] + row['stays_in_week_nights']
    for i in range(stay_duration):
        # Add 1 to the count for each day of the stay
        stay_date = row['arrival_date'] + pd.Timedelta(days=i)
        df.loc[(df['arrival_date'] == stay_date) &
               (df['reserved_room_type'] == row['reserved_room_type']),'count']+=1

In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:
finaldf['count'] = df['count']

In [ ]:
finaldf.head()

,arrival_date,weekend,adr,reserved_room_type,nights_staying,lead_time,adults,children,babies,meal,count
1,2015-07-01,0,0.0,C,0,737,2,0.0,0,BB,2
0,2015-07-01,0,0.0,C,0,342,2,0.0,0,BB,2
74966,2015-07-01,0,101.5,A,2,257,2,0.0,0,HB,98
74967,2015-07-01,0,101.5,A,2,257,2,0.0,0,HB,98
74968,2015-07-01,0,101.5,A,2,257,2,0.0,0,HB,98
...,...,...,...,...,...,...,...,...,...,...,...
73897,2015-07-02,0,62.0,A,2,258,2,0.0,0,BB,158
73898,2015-07-02,0,62.0,A,2,258,2,0.0,0,BB,158
73899,2015-07-02,0,62.0,A,2,258,2,0.0,0,BB,158
73900,2015-07-02,0,62.0,A,2,258,2,0.0,0,BB,158


In [ ]:
#Removing the rows in which the price is zero
finaldf = finaldf[pd.notna(finaldf['adr'])]
finaldf = finaldf[finaldf['adr']!= 0]


In [ ]:
#Getting the useful information from the arrival_date column

finaldf['arrival_year'] = finaldf['arrival_date'].dt.year
finaldf['arrival_month'] = finaldf['arrival_date'].dt.month
finaldf['arrival_day'] = finaldf['arrival_date'].dt.day
finaldf['arrival_day_of_week'] = finaldf['arrival_date'].dt.weekday
finaldf['arrival_day_of_year'] = finaldf['arrival_date'].dt.dayofyear

In [ ]:
finaldf = pd.get_dummies(finaldf, columns=['reserved_room_type','meal'])

In [ ]:
finaldf.drop(columns=["arrival_date"],inplace=True)

In [ ]:
finaldf = finaldf.astype('float64')

In [ ]:
finaldf.children = finaldf.children.fillna(0)

In [ ]:
from sklearn.model_selection import train_test_split

X = finaldf.drop(columns=['adr'])
y = finaldf['adr']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', max_depth=13, max_leaves=1200)

# Train the model
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=13, max_leaves=1200,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
mean_squared_error(y_test, y_pred)


1667.2420557317243